## Look at correlations between ASC and cross slope heat transport across different spatial scales (e.g. single basins)

In [1]:
import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib.path as mpath
import matplotlib.colors as col
import pandas as pd
import scipy.stats as stats
from tqdm import tqdm
import logging
import matplotlib.cm as cm
from dask.distributed import Client
import matplotlib.colors as col

logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

figdir = '/g/data/e14/txs156/Analysis/ACCESS-OM2/Clustering/ASC_and_heat_transport/Figures/'

In [2]:
client = Client(memory_limit='240gb',n_workers=48)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 48
Total threads: 48,Total memory: 10.48 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33587,Workers: 48
Dashboard: /proxy/8787/status,Total threads: 48
Started: Just now,Total memory: 10.48 TiB
Comm: tcp://127.0.0.1:39505,Total threads: 1
Dashboard: /proxy/33875/status,Memory: 223.52 GiB
Nanny: tcp://127.0.0.1:33747,


In [3]:
exp = '01deg_jra55v140_iaf_cycle3'
start_time= '1958-01-01'
end_time= '2018-12-31'
lat_range = slice(-90,-59)
isobath_depth = 1000

session = cc.database.create_session()

## First open u_along in density layers to use for compositing, and to show in time series:

In [4]:
# Load the along-slope velocity field
u_along = xr.open_mfdataset('/g/data/v45/wf4500/ASC_project_files/ASC_10deg_bin.nc')

# resample monthly:
u_along_da_months = u_along.resample(time='1M').mean()#.U_along_contour_binned
u_along_da_months = u_along_da_months.sel(potrho0=slice(13.8,28.42))#.rename({'lon_bin_midpoints': 'lon'})
u_along_da_months.load();

####################

# load monthly water mass mask:
age_mask = xr.open_mfdataset('/g/data/v45/wf4500/ASC_project_files/**_age_mask.nc')

# select out CDW layer:
u_along_da_months_binned_CDW = (age_mask.CDW_mask*u_along_da_months)

####################

# get volume for averaging across density layers:
vol_bins = xr.open_mfdataset('/g/data/v45/wf4500/ASC_project_files/Volume_rho_10deg_bins.nc')

# do volume-weighted density average and annual averaging:
def vol_ave_potrho0_potmask(quantity,vol_bins):
    return (quantity * vol_bins.volume).mean('potrho0')/vol_bins.mean('potrho0').volume

U_along_annual_binned_CDW_volav = vol_ave_potrho0_potmask(u_along_da_months_binned_CDW,vol_bins).groupby('time.year').mean('time')
U_along_annual_binned_CDW_volav = U_along_annual_binned_CDW_volav.load()
# climatological averaging and sum over layers:
U_along_clima_binned_CDW_volav = vol_ave_potrho0_potmask(u_along_da_months_binned_CDW,vol_bins).groupby('time.month').mean('time')
U_along_clima_binned_CDW_volav = U_along_clima_binned_CDW_volav.load()

## Then open CSHT in density layers for time series:

In [5]:
# Load CSHT field
CSHT_along = xr.open_mfdataset('/g/data/hh5/tmp/access-om/fbd581/ASC_project/Lon_binning/CSHT_ZC_potrho0_10degBins.nc')

# resample monthly:
CSHT_along_monthly = CSHT_along.resample(time='1M').mean()

# apply age mask to select out CDW layer:
CSHT_months_binned_CDW = (age_mask.CDW_mask * CSHT_along_monthly )

# annual averaging and sum over layers:
CSHT_annual_binned_CDW = CSHT_months_binned_CDW.groupby('time.year').mean('time').sum(dim = 'potrho0')
# climatological averaging and sum over layers:
CSHT_clima_binned_CDW = CSHT_months_binned_CDW.groupby('time.month').mean('time').sum(dim = 'potrho0')

## Now open correlation data to check we're selecting the correct longitude bin:

In [6]:
files_path = '/g/data/e14/qo9901/ASC_collab_CSHT_ds/'
file_extension = 'agemaskCDW_volav_corr'

ds = xr.open_dataset(files_path + 'r_sq_annual_10degBins_' + file_extension + '.nc')
r_sq_annual_10deg = ds.r_sq_annual
ds.close()

high_corr_lons = r_sq_annual_10deg.where(r_sq_annual_10deg>0.3,drop=True).lon_bins.values

## Find years when CDWlayer u_along is strongly westward (or less eastward):

In [7]:
longitude_slices = np.array([-63.5,-133.5,-243.5,-143.5,-173.5,-103.5, -33.5, -13.5, 76.5,-263.5])
## Check if high_correlation lons align with the regions we have selected:
print(np.isin(longitude_slices,high_corr_lons))
regions = ['West Antarctic Peninsula', 'Getz', 'Totten',\
           'Getz West', 'Glomar', 'Amundsen', 'Filchner', \
           'Riiser-Larsen', 'Amery', 'Denman']
min_lat = [-69, -74.25,-66.25, -75.25, -78,\
           -74.5, -78.5, -72, -69, -66]

vmin_range = np.array([-.2, -.05, -.05, -.05, -.05, \
                       -.05, -.05, -.05, -.05, -.05])
# for temp:
norm = [col.Normalize(vmin=-1.9,vmax=2.6),
col.Normalize(vmin=-1.8,vmax=1.5),
col.Normalize(vmin=-1.8,vmax=1.5),
col.Normalize(vmin=-1.8,vmax=2),
col.Normalize(vmin=-1.8,vmax=2),
col.Normalize(vmin=-1.8,vmax=2),
col.Normalize(vmin=-1.8,vmax=2),
col.Normalize(vmin=-1.8,vmax=2),
col.Normalize(vmin=-1.8,vmax=2),
col.Normalize(vmin=-1.8,vmax=2)]

levels=[np.arange(-1.9,2.6,.05), np.arange(-1.8,1.55,.05), np.arange(-1.8,1.55,.05),\
        np.arange(-1.8,2.05,.05),np.arange(-1.8,2.05,.05), np.arange(-1.8,2.05,.05),\
       np.arange(-1.8,2.05,.05),np.arange(-1.8,2.05,.05), np.arange(-1.8,2.05,.05),\
       np.arange(-1.8,2.05,.05)]
ticks = [[-1,0,1,2], [-1,0,1], [-1,0,1],
         [-1,0,1,2], [-1,0,1,2], [-1,0,1,2],
        [-1,0,1,2], [-1,0,1,2], [-1,0,1,2],
        [-1,0,1,2]]



[ True  True False  True  True  True  True  True  True  True]


In [8]:
# import bathymetry:
hu_transect_og = cc.querying.getvar(exp,'hu',session,n=1)
temp_og = cc.querying.getvar(exp, 'temp', session,frequency='1 monthly',start_time=start_time,end_time=end_time,chunks={'time': 12}) - 273.15
rho_og = cc.querying.getvar(exp, 'pot_rho_0', session,frequency='1 monthly',start_time=start_time,end_time=end_time,chunks={'time': 12})
u_og = cc.querying.getvar(exp, 'u', session,frequency='1 monthly',start_time=start_time,end_time=end_time,chunks={'time': 12})

In [9]:
##########################
hu_transect_list = []
hu_sliced_list = []
max_lat = []
southward_threshold = []
northward_threshold = []
temp_southward = []
temp_northward = []
rho_southward = []
rho_northward = []
u_southward = []
u_northward = [s


for i in tqdm(range(longitude_slices.size)):
    print(i)
    # select out longitude:
    u_along_regional = U_along_annual_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])
    
    # find u_along threshold for most southward 100 months:
    u_sorted = u_along_regional.sortby(u_along_regional)
    npoints = 10
    southward_threshold.append(u_sorted[npoints].data) #negative, westward threshold for ASC
    northward_threshold.append(u_sorted[-npoints].data) #Positive, eastward threshold for ASC

    ## NOTE: We are now averaging over ten longitude bins

    # import bathymetry:
    hu_sliced = hu_transect_og.sel(xu_ocean=longitude_slices[i], method='nearest').sel(yu_ocean=lat_range)
    hu_transect = hu_transect_og.sel(xu_ocean=slice(longitude_slices[i]-5, longitude_slices[i]+5)).mean('xu_ocean').sel(yu_ocean=lat_range)

    # fill nans with zeros:
    hu_transect = hu_transect.where(hu_transect>0,0)
    hu_sliced = hu_sliced.where(hu_sliced>0,0)
    hu_transect_list.append(hu_transect)
    hu_sliced_list.append(hu_sliced)
    
    # find latitude limits for plot:
    # lats = (hu_transect.yu_ocean).where(hu_transect>0).compute()
    #min_lat = lats.dropna('yu_ocean').data[0]-.1
    #lats = (hu_transect.yu_ocean).where(hu_transect>3000).compute()
    lats = (hu_sliced.yu_ocean).where(hu_sliced>3000).compute()
    max_lat.append(lats.dropna('yu_ocean').data[0])
    

    # Use monthly data to pick southward times:
    temp = temp_og.sel(time=slice(start_time,end_time)).sel(xt_ocean=longitude_slices[i], method='nearest')\
                                                                       .sel(st_ocean=slice(0,1300)).sel(yt_ocean=lat_range)
    temp = temp.groupby('time.year').mean().load()
    
    temp_southward.append(temp.where(u_along_regional < southward_threshold[i]).mean('year').load())
    temp_northward.append(temp.where(u_along_regional > northward_threshold[i]).mean('year').load())
    
    rho = rho_og.sel(time=slice(start_time,end_time)).sel(xt_ocean=longitude_slices[i], method='nearest')\
                                                                       .sel(st_ocean=slice(0,1300)).sel(yt_ocean=lat_range)
    rho = rho.groupby('time.year').mean().load()
    
    rho_southward.append(rho.where(u_along_regional < southward_threshold[i]).mean('year').load())
    rho_northward.append(rho.where(u_along_regional > northward_threshold[i]).mean('year').load())
    
    u = u_og.sel(time=slice(start_time,end_time)).sel(xu_ocean=longitude_slices[i], method='nearest')\
                                                                       .sel(st_ocean=slice(0,1300)).sel(yu_ocean=lat_range)
    u = u.groupby('time.year').mean().load()
    
    u_southward.append(u.where(u_along_regional < southward_threshold[i]).mean('year').load())
    u_northward.append(u.where(u_along_regional > northward_threshold[i]).mean('year').load())

SyntaxError: '[' was never closed (3864529209.py, line 12)

In [ ]:
arr = np.linspace(-0.03, 0.03, 7)  # Generates 5 points
arr = arr[arr != 0]  # Remove zero
arr

u_levels = [[-.24,-.20,-.16,-.12,-.08,-.04,.04,.08,.12,.16,.20,.24],
            arr,
            [-.03,-.015,.015,.03],
            arr,
            arr,
            [-.06,-.04,-.02,.02,.04,.06],
            [-.06,-.04,-.02,.02,.04,.06],
            [-.06,-.04,-.02,.02,.04,.06],
            [-.06,-.04,-.02,.02,.04,.06],
            [-.06,-.04,-.02,.02,.04,.06]
           ]


density_contours = [[1027.65,1027.78,1027.8],
                    [1027.65,1027.78,1027.8,1027.82,1027.83],
                    [1027.65,1027.78,1027.8,1027.82,1027.83],
                    [1027.65,1027.78,1027.8,1027.82,1027.83],
                    [1027.65,1027.78,1027.8,1027.82,1027.83],
                    [1027.65,1027.78,1027.8,1027.82,1027.83]]
                    


In [ ]:

# Function x**(1/2)
def forward(x):
    return x**(1/2)


def inverse(x):
    return x**2

In [ ]:
##### UNCOMMENT BELOW TO SEE ALL REGIONS #####

# # plot single longitude:
# # plot:
# for i in range(np.array(regions).size):
#     fig=plt.figure(figsize=(7,6))
#     plt.clf()
    
#     # Northward CSHT:
#     ax1 = fig.add_axes([0.0,0.5,0.9,0.45])
    
#     pcm = ax1.contourf(temp_northward[i].yt_ocean,temp_northward[i].st_ocean,
#                        temp_northward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),
#                        cmap = cmocean.thermal, norm = norm[i],levels=levels[i],extend='both')
#     #ax1.fill_between(temp_southward[i].yt_ocean,hu_transect_list[i],3000,zorder=3,color='darkgrey')
#     ax1.fill_between(temp_southward[i].yt_ocean,hu_sliced_list[i],3000,zorder=3,color='darkgrey')

#     ax1.set_yscale('function', functions=(forward, inverse))
#     ax1.set_ylim(1200,0.5)
#     ax1.set_xlim(min_lat[i]-1,max_lat[i]+1)
#     ax1.set_ylabel('Depth (m)',fontsize=fontsize)
#     #ax1.set_xlabel('Longitude',fontsize=fontsize)
#     ax1.set_xticks([])
#     ax1.set_yticks([1200,800,400,100,0.5],labels=[1200,800,400,100,0])
#     plt.tick_params(labelsize=fontsize)
#     ax1.set_title(str(regions[i])+' ('+str(longitude_slices[i])+')\n r-squared: '+str(r_sq_annual_10deg.sel(lon_bins=longitude_slices[i]).values.round(2)))
    
#     # colorbar:
#     cax = fig.add_axes([0.94,0.5,0.022,0.45])
#     cbar = plt.colorbar(pcm, cax=cax,orientation = 'vertical',extend = 'neither',ticks=ticks[i])
#     cbar.set_label(r'Temperature ($^\circ$C)',fontsize=fontsize,labelpad=5)
#     cbar.ax.tick_params(labelsize=fontsize)
    
#     cont=ax1.contour(rho_northward[i].yt_ocean,rho_northward[i].st_ocean,
#                      rho_northward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),density_contours,
#                       colors='k',zorder=2 ,fontsize=fontsize,linestyles='--')
    
#     cont2=ax1.contour(u_northward[i].yu_ocean,u_northward[i].st_ocean,
#                      u_northward[i].interpolate_na(dim='yu_ocean',method='nearest',fill_value='extrapolate'),u_levels[i],vmin=vmin_range[i],vmax=-vmin_range[i],
#                       cmap=cmocean.delta,zorder=2 ,fontsize=fontsize)
    
#     cont2=ax1.pcolormesh(u_northward[i].yu_ocean,u_northward[i].st_ocean,
#                      u_northward[i].interpolate_na(dim='yu_ocean',method='nearest',fill_value='extrapolate'),vmin=vmin_range[i],vmax=-vmin_range[i],
#                       cmap=cmocean.delta,zorder=0 )
    
#     ax1.text(min_lat[i]-0.99, 1100, '10 weakest\nslope current\nyears', fontsize=15)
#     ####################
    
#     ax2 = fig.add_axes([0.0,0.0,0.9,0.45])
    
#     pcm = ax2.contourf(temp_southward[i].yt_ocean,temp_southward[i].st_ocean,
#                        temp_southward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),
#                        cmap = cmocean.thermal, norm = norm[i],levels=levels[i],extend='both')
#     # ax2.fill_between(temp_southward[i].yt_ocean,hu_transect_list[i],3000,zorder=3,color='darkgrey')
#     ax2.fill_between(temp_southward[i].yt_ocean,hu_sliced_list[i],3000,zorder=3,color='darkgrey')

#     ax2.set_yscale('function', functions=(forward, inverse))
#     ax2.set_ylim(1200,0.5)
#     ax2.set_xlim(min_lat[i]-1,max_lat[i]+1)
#     ax2.set_ylabel('Depth (m)',fontsize=fontsize)
#     ax2.set_xlabel('Latitude',fontsize=fontsize)
#     #ax2.set_xticks([-74,-73.5,-73,-72.5])
#     #ax2.set_xticks([-66.5,-66,-65.5,-65])
#     ax2.set_yticks([1200,800,400,100,0.5],labels=[1200,800,400,100,0])
#     plt.tick_params(labelsize=fontsize)
#     ax2.text(min_lat[i]-0.99, 1100, '10 strongest\nslope current\nyears', fontsize=15)
    
#     #ax2.set_title('u_aong westward (or less northward)',fontsize=fontsize)
    
#     # colorbar:
#     cax = fig.add_axes([0.94,0.0,0.022,0.45])
#     cbar = plt.colorbar(cont2, cax=cax,orientation = 'vertical',extend = 'neither',ticks=ticks[i])
#     cbar.set_label(r'Velocity (m/s)',fontsize=fontsize,labelpad=5)
#     cbar.ax.tick_params(labelsize=fontsize)
    
    
#     cont=ax2.contour(rho_southward[i].yt_ocean,rho_southward[i].st_ocean,
#                      rho_southward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),density_contours,
#                       colors='k',zorder=2 ,fontsize=fontsize,linestyles='--')
    
#     cont2=ax2.contour(u_southward[i].yu_ocean,u_southward[i].st_ocean,
#                      u_southward[i].interpolate_na(dim='yu_ocean',method='nearest',fill_value='extrapolate'),u_levels[i],vmin=vmin_range[i],vmax=-vmin_range[i],
#                       cmap=cmocean.delta,zorder=2 ,fontsize=fontsize)
    
    
#     ###################
    
#     ax3 = fig.add_axes([0.0,-0.5,0.9,0.4])
#     ax3_HT=ax3.twinx()
#     ax3.plot(U_along_annual_binned_CDW_volav.year,U_along_annual_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])*100,label='ASC')
#     ax3.plot([1956,2024],[0,0],\
#              linestyle='--',color='black')
#     ax3_HT.plot(U_along_annual_binned_CDW_volav.year,(CSHT_annual_binned_CDW.CSHT_ZC_lon_binned.sel(lon_bins=longitude_slices[i])/1e12),\
#                 label='CSHT',color='tab:orange')
#     # ax3.set_ylim(u_along_lims[i])
#     # ax3.set_yticks([-3,-2,-1,0,1,2,3],labels=[-3,-2,-1,0,1,2,3],fontsize=14,color='tab:blue')
#     ax3.set_ylabel('ASC speed (cm/s)',fontsize=fontsize,color='tab:blue')
#     # ax3_HT.set_ylim(CSHT_lims[i])
#     # ax3_HT.set_yticks([-10,-5,0,5,10],labels=[-10,-5,0,5,10],fontsize=14,color='tab:orange')
#     ax3_HT.set_ylabel('Cross Slope Heat Transport \n (TW)',fontsize=fontsize,color='tab:orange')
    
#     ax3.set_xticks(ticks=np.arange(1960,2021,10),labels=np.arange(1960,2021,10), fontsize=14)
#     ax3.set_xlim(1957,2020)
#     ax3.set_xlabel('Year')
#     ax3.text(1959, np.nanmin(U_along_annual_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])*100)+0.01, 'Interannual', fontsize=15)
    
#     ################
    
#     ax4 = fig.add_axes([0.0,-1,0.9,0.4])
#     ax4_HT=ax4.twinx()
#     ax4.plot(U_along_clima_binned_CDW_volav.month,U_along_clima_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])*100,label='ASC')
#     ax4.plot([1956,2024],[0,0],\
#              linestyle='--',color='black')
#     ax4_HT.plot(U_along_clima_binned_CDW_volav.month,(CSHT_clima_binned_CDW.CSHT_ZC_lon_binned.sel(lon_bins=longitude_slices[i])/1e12),\
#                 label='CSHT',color='tab:orange')
#     # ax3.set_ylim(u_along_lims[i])
#     # ax3.set_yticks([-3,-2,-1,0,1,2,3],labels=[-3,-2,-1,0,1,2,3],fontsize=14,color='tab:blue')
#     ax4.set_ylabel('ASC speed (cm/s)',fontsize=fontsize,color='tab:blue')
#     # ax3_HT.set_ylim(CSHT_lims[i])
#     # ax3_HT.set_yticks([-10,-5,0,5,10],labels=[-10,-5,0,5,10],fontsize=14,color='tab:orange')
#     ax4_HT.set_ylabel('Cross Slope Heat Transport \n (TW)',fontsize=fontsize,color='tab:orange')
#     ax4.set_xlabel('Month')
#     # ax4.set_xticks(ticks=np.arange(1960,2021,10),labels=np.arange(1960,2021,10), fontsize=14)
#     ax4.set_xlim(1,12)
#     ax4.text(1.25, np.nanmin(U_along_clima_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])*100)+0.01, 'Climatology', fontsize=15)
    
#     # save:
#     figure_path = figdir+'Transect_'+str(regions[i])+'.png'
#     plt.savefig(figure_path , dpi=300,bbox_inches='tight', pad_inches=0.1,facecolor='white')
#     plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs

fontsize = 12

regions_selected = ['Getz', 'West Antarctic Peninsula', 'Glomar']
# Find indices where regions match those in regions_selected
inds_regions = np.where(np.isin(regions, regions_selected))[0]
min_lats_selected = [-63.5, -74.6, -77]
# Create a figure
fig = plt.figure(figsize=(16*0.7, 8*0.7))

# Create a GridSpec for a 3x4 grid
grid = gs.GridSpec(nrows=3, ncols=3, figure=fig)
grid.update(hspace=0.2, wspace=0.27)  # Adjust vertical and horizontal spacing

# Initialize a list to store axes
axes = []

subplots = ['a)', 'b)', 'c)', 'd)', 'e)', 'f)', 'g)', 'h)', 'i)']

# Loop through the 3x4 grid to create subplots
for j in range(inds_regions.size):  # Loop over rows
    ax1 = fig.add_subplot(grid[0, j])
    ax2 = fig.add_subplot(grid[1, j])
    ax3 = fig.add_subplot(grid[2, j])

    i = inds_regions[j]
    print(i)
    pcm = ax1.contourf(temp_northward[i].yt_ocean,temp_northward[i].st_ocean,
                       temp_northward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),
                       cmap = cmocean.thermal, norm = norm[i],levels=levels[i],extend='both')
    #ax1.fill_between(temp_southward[i].yt_ocean,hu_transect_list[i],3000,zorder=3,color='darkgrey')
    ax1.fill_between(temp_southward[i].yt_ocean,hu_sliced_list[i],3000,zorder=3,color='darkgrey')

    ax1.set_yscale('function', functions=(forward, inverse))
    ax1.set_ylim(1200,0.5)
    ax1.set_xlim(min_lats_selected[j],max_lat[i]+2)
    if j == 0:
        ax1.set_ylabel('Depth (m)',fontsize=fontsize)
    #ax1.set_xlabel('Longitude',fontsize=fontsize)
    ax1.set_xticks([])
    ax1.set_yticks([1200,800,400,100,0.5],labels=[1200,800,400,100,0])
    ax1.set_title(str(regions[i])+' ('+str(abs(longitude_slices[i]))+' $^{\circ}$W'+')\n r$^2$: '+str(r_sq_annual_10deg.sel(lon_bins=longitude_slices[i]).values.round(2)))
    
    # colorbar:
    
    cont=ax1.contour(rho_northward[i].yt_ocean,rho_northward[i].st_ocean,
                     rho_northward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),density_contours[i],
                      colors='k',zorder=2 ,fontsize=fontsize,linestyles='-')
    
    cont2=ax1.contour(u_northward[i].yu_ocean,u_northward[i].st_ocean,
                     u_northward[i].interpolate_na(dim='yu_ocean',method='nearest',fill_value='extrapolate'),u_levels[i],vmin=vmin_range[i],vmax=-vmin_range[i],
                      cmap=cmocean.delta,zorder=2 ,fontsize=fontsize)
    
    cont2=ax1.pcolormesh(u_northward[i].yu_ocean,u_northward[i].st_ocean,
                     u_northward[i].interpolate_na(dim='yu_ocean',method='nearest',fill_value='extrapolate'),vmin=vmin_range[i],vmax=-vmin_range[i],
                      cmap=cmocean.delta,zorder=0 )

    ax1.tick_params(labelsize=fontsize)
    ####################
        
    pcm = ax2.contourf(temp_southward[i].yt_ocean,temp_southward[i].st_ocean,
                       temp_southward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),
                       cmap = cmocean.thermal, norm = norm[i],levels=levels[i],extend='both')
    # ax2.fill_between(temp_southward[i].yt_ocean,hu_transect_list[i],3000,zorder=3,color='darkgrey')
    ax2.fill_between(temp_southward[i].yt_ocean,hu_sliced_list[i],3000,zorder=3,color='darkgrey')

    ax2.set_yscale('function', functions=(forward, inverse))
    ax2.set_ylim(1200,0.5)
    ax2.set_xlim(min_lats_selected[j],max_lat[i]+2)
    ax2.set_xticks([])

    if j == 0:
        ax2.set_ylabel('Depth (m)',fontsize=fontsize)
    # ax2.set_xlabel('Latitude',fontsize=fontsize)
    #ax2.set_xticks([-74,-73.5,-73,-72.5])
    #ax2.set_xticks([-66.5,-66,-65.5,-65])
    ax2.set_yticks([1200,800,400,100,0.5],labels=[1200,800,400,100,0])
    
    #ax2.set_title('u_aong westward (or less northward)',fontsize=fontsize)    
    
    cont=ax2.contour(rho_southward[i].yt_ocean,rho_southward[i].st_ocean,
                     rho_southward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),density_contours[i],
                      colors='k',zorder=2 ,fontsize=fontsize,linestyles='--')
    
    cont2=ax2.contour(u_southward[i].yu_ocean,u_southward[i].st_ocean,
                     u_southward[i].interpolate_na(dim='yu_ocean',method='nearest',fill_value='extrapolate'),u_levels[i],vmin=vmin_range[i],vmax=-vmin_range[i],
                      cmap=cmocean.delta,zorder=2 ,fontsize=fontsize)
    ax2.tick_params(labelsize=fontsize)
    #########

    pci = ax3.contourf(temp_southward[i].yt_ocean,temp_southward[i].st_ocean,
                   temp_northward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate')-\
                    temp_southward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),
                   cmap = cm.bwr,levels=np.linspace(-1,1,200), vmin=-1, vmax=1, extend='both',fontsize=fontsize)
    ax3.fill_between(temp_southward[i].yt_ocean,hu_sliced_list[i],3000,zorder=3,color='darkgrey')

    ax3.set_yscale('function', functions=(forward, inverse))
    ax3.set_ylim(1200,0.5)
    ax3.set_xlim(min_lats_selected[j],max_lat[i]+2)
    if j == 0:
        ax3.set_ylabel('Depth (m)',fontsize=fontsize)
    ax3.set_xlabel('Latitude',fontsize=fontsize)
    ax3.set_yticks([1200,800,400,100,0.5],labels=[1200,800,400,100,0])
    ax3.tick_params(labelsize=fontsize)

    cont=ax3.contour(rho_northward[i].yt_ocean,rho_northward[i].st_ocean,
                     rho_northward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),density_contours[i],
                      colors='k',zorder=2 ,fontsize=fontsize,linestyles='-')
    cont=ax3.contour(rho_southward[i].yt_ocean,rho_southward[i].st_ocean,
                     rho_southward[i].interpolate_na(dim='yt_ocean',method='nearest',fill_value='extrapolate'),density_contours[i],
                      colors='k',zorder=2 ,fontsize=fontsize,linestyles='--')

    ax1.text(-0.15,1, subplots[j], fontsize=fontsize, transform = ax1.transAxes, va = 'center', ha = 'center')
    ax2.text(-0.15,1, subplots[j+3], fontsize=fontsize, transform = ax2.transAxes, va = 'center', ha = 'center')
    ax3.text(-0.15,1, subplots[j+6], fontsize=fontsize, transform = ax3.transAxes, va = 'center', ha = 'center')

cax = fig.add_axes([0.93,0.43,0.022,0.43])
cbar = plt.colorbar(pcm, cax=cax,orientation = 'vertical',extend = 'neither',ticks=ticks[i])
cbar.set_label(r'Temperature ($^\circ$C)',fontsize=fontsize,labelpad=5)
cbar.ax.tick_params(labelsize=fontsize)

cax = fig.add_axes([0.93,0.095,0.022,0.25])
cbar = plt.colorbar(pci, cax=cax,orientation = 'vertical',extend = 'neither',ticks=[-1,0,1])
cbar.set_label(r'Difference ($^\circ$C)',fontsize=fontsize,labelpad=5)
cbar.ax.tick_params(labelsize=fontsize)

# colorbar:
sm = plt.cm.ScalarMappable(cmap=cmocean.delta, norm=plt.Normalize(vmin=-.24, vmax = 0.24))

cax = fig.add_axes([0.13,-0.04,0.2,0.022])
cbar = plt.colorbar(sm, cax=cax,orientation = 'horizontal',extend = 'neither')
cbar.set_label(r'Velocity (m/s)',fontsize=fontsize,labelpad=5)
cbar.ax.tick_params(labelsize=fontsize)

# colorbar:

sm = plt.cm.ScalarMappable(cmap=cmocean.delta, norm=plt.Normalize(vmin=-.03, vmax = 0.03))

cax = fig.add_axes([0.4,-0.04,0.5,0.022])
cbar = plt.colorbar(sm, cax=cax,orientation = 'horizontal',extend = 'neither')
cbar.set_label(r'Velocity (m/s)',fontsize=fontsize,labelpad=5)
cbar.ax.tick_params(labelsize=fontsize)
ax1.text(-3.05,0.5, 'Eastward\nASC Anomaly', fontsize=fontsize, transform = ax1.transAxes, va = 'center', ha = 'center', rotation = 90)
ax2.text(-3.05,0.5, 'Westward\nASC Anomaly', fontsize=fontsize, transform = ax2.transAxes, va = 'center', ha = 'center', rotation = 90)
ax3.text(-3.05,0.5, 'Eastward\nminus\nwestward', fontsize=fontsize, transform = ax3.transAxes, va = 'center', ha = 'center', rotation = 90)


# Adjust layout to avoid overlap
plt.tight_layout()
figure_path = figdir+'selected_regions_transects.png'
plt.savefig(figure_path , dpi=300,bbox_inches='tight', pad_inches=0.1,facecolor='white')
# Show the plots
plt.show()

In [ ]:
regions_selected = ['Getz East', 'West Antarctic Peninsula', 'Glomar']

inds_regions = np.where(np.isin(regions, regions_selected))[0]

fig = plt.figure(figsize=(16*0.7, 16*0.7))

# Create a GridSpec for a 3x4 grid
grid = gs.GridSpec(nrows=2, ncols=3, figure=fig)
grid.update(hspace=0.4, wspace=0.35)  # Adjust vertical and horizontal spacing

# Initialize a list to store axes
axes = []

# Loop through the 3x4 grid to create subplots
for j in range(inds_regions.size):  # Loop over rows

    
    i = inds_regions[j]

    ax3 = fig.add_subplot(grid[0, j])
    ax4 = fig.add_subplot(grid[1, j])
    ###################
    
    ax3_HT=ax3.twinx()
    ax3.plot(U_along_annual_binned_CDW_volav.year,U_along_annual_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])*100,label='ASC')
    ax3.plot([1956,2024],[0,0],\
             linestyle='--',color='black')
    ax3_HT.plot(U_along_annual_binned_CDW_volav.year,(CSHT_annual_binned_CDW.CSHT_ZC_lon_binned.sel(lon_bins=longitude_slices[i])/1e12),\
                label='CSHT',color='tab:orange')
    # ax3.set_ylim(u_along_lims[i])
    # ax3.set_yticks([-3,-2,-1,0,1,2,3],labels=[-3,-2,-1,0,1,2,3],fontsize=14,color='tab:blue')
    if j == 0:    
        ax3.set_ylabel('ASC speed (cm/s)',fontsize=fontsize,color='tab:blue')
    # ax3_HT.set_ylim(CSHT_lims[i])
    # ax3_HT.set_yticks([-10,-5,0,5,10],labels=[-10,-5,0,5,10],fontsize=14,color='tab:orange')
    if j == inds_regions.size-1:
        ax3_HT.set_ylabel('Cross Slope\nHeat Transport \n (TW)',fontsize=fontsize,color='tab:orange')
    
    ax3.set_xticks(ticks=np.arange(1960,2021,20),labels=np.arange(1960,2021,20), fontsize=fontsize)
    ax3.grid()
    ax3.set_xlim(1957,2020)
    ax3.set_xlabel('Year')
    ax3.tick_params(axis='y', colors='tab:blue')  # Left y-axis ticks and labels in blue
    ax3_HT.tick_params(axis='y', colors='tab:orange')  # Right y-axis ticks and labels in orange

    ################
    
    ax4_HT=ax4.twinx()
    ax4.plot(U_along_clima_binned_CDW_volav.month,U_along_clima_binned_CDW_volav.U.sel(lon_bins=longitude_slices[i])*100,label='ASC')
    ax4.plot([0,12],[0,0],\
             linestyle='--',color='black')
    ax4_HT.plot(U_along_clima_binned_CDW_volav.month,(CSHT_clima_binned_CDW.CSHT_ZC_lon_binned.sel(lon_bins=longitude_slices[i])/1e12),\
                label='CSHT',color='tab:orange')
    # ax3.set_ylim(u_along_lims[i])
    # ax3.set_yticks([-3,-2,-1,0,1,2,3],labels=[-3,-2,-1,0,1,2,3],fontsize=14,color='tab:blue')
    if j == 0:    
        ax4.set_ylabel('ASC speed (cm/s)',fontsize=fontsize,color='tab:blue')
    # ax3_HT.set_ylim(CSHT_lims[i])
    # ax3_HT.set_yticks([-10,-5,0,5,10],labels=[-10,-5,0,5,10],fontsize=14,color='tab:orange')
    if j == inds_regions.size-1:
        ax4_HT.set_ylabel('Cross Slope\nHeat Transport \n (TW)',fontsize=fontsize,color='tab:orange')
    ax4.set_xlabel('Month')
    # ax4.set_xticks(ticks=np.arange(1960,2021,10),labels=np.arange(1960,2021,10), fontsize=14)
    ax4.set_xticks(ticks=np.arange(1,12,2),labels=np.arange(1,12,2), fontsize=fontsize)
    ax4.set_xlim(1,12)
    ax4.grid()
    # Set tick colors
    ax4.tick_params(axis='y', colors='tab:blue')  # Left y-axis ticks and labels in blue
    ax4_HT.tick_params(axis='y', colors='tab:orange')  # Right y-axis ticks and labels in orange

ax3.text(-3.2,0.5, 'Interannual', fontsize=fontsize, transform = ax3.transAxes, va = 'center', ha = 'center', rotation = 90)
ax4.text(-3.2,0.5, 'Climatology', fontsize=fontsize, transform = ax4.transAxes, va = 'center', ha = 'center', rotation = 90)

plt.tight_layout()
figure_path = figdir+'selected_time_series.png'
plt.savefig(figure_path , dpi=300,bbox_inches='tight', pad_inches=0.1,facecolor='white')
# Show the plots
plt.show()